In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [43]:
raw_data = pd.read_csv("/Users/schlinkertc/Flatiron/projects/mod4_project/developer_survey_2019/survey_results_public.csv")
raw_data.sort_values('ConvertedComp',axis=0,ascending=False)[:50]

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
25376,25524,I am a developer by profession,Yes,Less than once per year,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers,33.0,Man,No,Straight / Heterosexual,Hispanic or Latino/Latina,NaN,Appropriate in length,Neither easy nor difficult
27847,28008,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area,28.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
84868,85395,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,United States,No,Associate degree,I never declared a major,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
44271,44552,I am a developer by profession,No,Never,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Tech...,26.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Neither easy nor difficult
68771,69211,"I am not primarily a developer, but I write co...",No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...",...,Just as welcome now as I felt last year,NaN,23.0,Man,No,Straight / Heterosexual,White or of European descent,No,Too long,Neither easy nor difficult
87814,88362,I am a developer by profession,No,Less than once per year,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Industry news about technologies you're intere...,40.0,Man,No,Straight / Heterosexual,Black or of African descent,No,Appropriate in length,Easy
55110,55459,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Other doctoral degree (Ph.D, Ed.D., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area,42.0,Man,No,Straight / Heterosexual,White or of European descent,Yes,Too long,Neither easy nor difficult
55106,55455,I am a developer by profession,Yes,Never,"OSS is, on average, of LOWER quality than prop...",Employed full-time,United States,"Yes, full-time","Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,28.0,Man,No,Straight / Heterosexual,South Asian,No,Too long,Easy
18884,18987,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,52.0,Man,No,Straight / Heterosexual,White or o

In [3]:
col_less12 = []
for x, y in [(x, raw_data[x].value_counts()) for x in raw_data.columns[1:]]:
    if len(y)<12:
        col_less12.append(x)
        print(x,len(y),y,sep='\n')
col_less12

MainBranch
5
I am a developer by profession                                                   65679
I am a student who is learning to code                                           10189
I am not primarily a developer, but I write code sometimes as part of my work     7539
I code primarily as a hobby                                                       3340
I used to be a developer by profession, but no longer am                          1584
Name: MainBranch, dtype: int64
Hobbyist
2
Yes    71257
No     17626
Name: Hobbyist, dtype: int64
OpenSourcer
4
Never                                                 32295
Less than once per year                               24972
Less than once a month but more than once per year    20561
Once a month or more often                            11055
Name: OpenSourcer, dtype: int64
OpenSource
3
The quality of OSS and closed source software is about the same                    41527
OSS is, on average, of HIGHER quality than proprietary / closed sou

['MainBranch',
 'Hobbyist',
 'OpenSourcer',
 'OpenSource',
 'Employment',
 'Student',
 'EdLevel',
 'OrgSize',
 'CareerSat',
 'JobSat',
 'MgrIdiot',
 'MgrMoney',
 'MgrWant',
 'JobSeek',
 'LastHireDate',
 'FizzBuzz',
 'ResumeUpdate',
 'CompFreq',
 'WorkPlan',
 'WorkRemote',
 'WorkLoc',
 'ImpSyn',
 'CodeRev',
 'UnitTests',
 'PurchaseHow',
 'PurchaseWhat',
 'OpSys',
 'BlockchainOrg',
 'BlockchainIs',
 'BetterLife',
 'ITperson',
 'OffOn',
 'Extraversion',
 'ScreenName',
 'SOVisitFreq',
 'SOFindAnswer',
 'SOTimeSaved',
 'SOHowMuchTime',
 'SOAccount',
 'SOPartFreq',
 'SOJobs',
 'EntTeams',
 'SOComm',
 'WelcomeChange',
 'Gender',
 'Trans',
 'Sexuality',
 'Dependents',
 'SurveyLength',
 'SurveyEase']

In [4]:
len(col_less12)

50

In [5]:
df = (raw_data[['Respondent','ConvertedComp','YearsCode','Age','OrgSize','Gender','Ethnicity','LanguageWorkedWith','Country','WorkWeekHrs','Extraversion','Dependents']].dropna())


In [6]:
df.shape

(47083, 12)

In [7]:
df = df.replace(to_replace =["Less than 1 year","More than 50 years"], value =["0","50"])
df['YearsCode'] = pd.to_numeric(df['YearsCode'])

In [8]:
df['LanguageWorkedWith'] = [x.split(sep=';') for x in df['LanguageWorkedWith'].values]
df_language = pd.get_dummies(df['LanguageWorkedWith'].apply(pd.Series).stack()).sum(level=0)

df = df.join(df_language)
df = df.drop(labels='LanguageWorkedWith',axis=1)

In [9]:
df['Ethnicity'] = [x.split(sep=';') for x in df['Ethnicity'].values]
df_ethnicity = pd.get_dummies(df['Ethnicity'].apply(pd.Series).stack()).sum(level=0)

df = df.join(df_ethnicity)
df = df.drop(labels='Ethnicity',axis=1)

In [10]:
# def dummy(df):
#     for x in df.columns:
#         try:
#             #display(df[x].head())
#             df[x] = [y.split(sep=';') for y in df[x].values]
#             #df_temp = pd.get_dummies(df[x].apply(pd.Series).stack()).sum(level=0)
#             #df.join(df_temp)
#             #df.drop(labels=x,axis=1)
#             print('worked for: ',x)
#         except:
#             print("didn't work: ", x)
#     return df
        

In [11]:
# def dummy(df,column):
#     try:
#         df[column] = [x.split(sep=';') for x in df[column].values]
#     except:
#         None
#     try:
#         df_temp = pd.get_dummies(df[column].apply(pd.Series).stack()).sum(level=0)
#         df = df.join(df_temp)
#         df = df.drop(labels=column,axis=1)
#     except: 
#         None
#     return df

In [12]:
df['Gender'].unique()

array(['Man', 'Woman',
       'Non-binary, genderqueer, or gender non-conforming',
       'Woman;Non-binary, genderqueer, or gender non-conforming',
       'Man;Non-binary, genderqueer, or gender non-conforming',
       'Woman;Man',
       'Woman;Man;Non-binary, genderqueer, or gender non-conforming'],
      dtype=object)

In [13]:
df.loc[df['Gender'].str.contains(";"),'Gender']='Non-binary, genderqueer, or gender non-conforming'

In [14]:
df['Gender'].unique()

array(['Man', 'Woman',
       'Non-binary, genderqueer, or gender non-conforming'], dtype=object)

In [15]:
df = pd.get_dummies(df)

In [33]:
df['ConvertedComp']

3          61000.0
8          95179.0
12         90000.0
13         57060.0
15        455352.0
           ...    
88324    2000000.0
88325     130000.0
88326      82488.0
88328      68745.0
88330      22915.0
Name: ConvertedComp, Length: 47083, dtype: float64

In [42]:
df.sort_values('ConvertedComp',axis=0,ascending=False)[:50]

,Respondent,ConvertedComp,YearsCode,Age,WorkWeekHrs,Assembly,Bash/Shell/PowerShell,C,C#,C++,...,"Country_Venezuela, Bolivarian Republic of...",Country_Viet Nam,Country_Yemen,Country_Zambia,Country_Zimbabwe,Extraversion_In real life (in person),Extraversion_Neither,Extraversion_Online,Dependents_No,Dependents_Yes
57307,57673,2000000.0,8,26.0,40.0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
77553,78039,2000000.0,2,27.0,45.0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
35386,35605,2000000.0,8,28.0,55.0,0,1,1,1,1,...,0,0,0,0,0,1,0,0,1,0
44157,44437,2000000.0,16,32.0,40.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3251,3265,2000000.0,14,22.0,40.0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
3269,3283,2000000.0,15,29.0,50.0,0,1,0,1,1,...,0,0,0,0,0,1,0,0,1,0
32838,33041,2000000.0,30,40.0,50.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
56964,57325,2000000.0,6,21.0,40.0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,1,0
17988,18088,2000000.0,29,43.0,50.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
60719,61109,2000000.0,30,41.0,40.0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,1


In [17]:
list(df.columns)

['Respondent',
 'ConvertedComp',
 'YearsCode',
 'Age',
 'WorkWeekHrs',
 'Assembly',
 'Bash/Shell/PowerShell',
 'C',
 'C#',
 'C++',
 'Clojure',
 'Dart',
 'Elixir',
 'Erlang',
 'F#',
 'Go',
 'HTML/CSS',
 'Java',
 'JavaScript',
 'Kotlin',
 'Objective-C',
 'Other(s):',
 'PHP',
 'Python',
 'R',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Swift',
 'TypeScript',
 'VBA',
 'WebAssembly',
 'Biracial',
 'Black or of African descent',
 'East Asian',
 'Hispanic or Latino/Latina',
 'Middle Eastern',
 'Multiracial',
 'Native American, Pacific Islander, or Indigenous Australian',
 'South Asian',
 'White or of European descent',
 'OrgSize_1,000 to 4,999 employees',
 'OrgSize_10 to 19 employees',
 'OrgSize_10,000 or more employees',
 'OrgSize_100 to 499 employees',
 'OrgSize_2-9 employees',
 'OrgSize_20 to 99 employees',
 'OrgSize_5,000 to 9,999 employees',
 'OrgSize_500 to 999 employees',
 'OrgSize_Just me - I am a freelancer, sole proprietor, etc.',
 'Gender_Man',
 'Gender_Non-binary, genderqueer, or gender

In [18]:
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import sklearn.preprocessing as preprocessing

In [19]:
sklearn.model_selection.GroupKFold()

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:503: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


GroupKFold(n_splits=3)

In [20]:
regr = linear_model.LinearRegression()

In [21]:
X = df.drop(columns='ConvertedComp')
Y = df['ConvertedComp']

regr.fit(X, Y)

#print('Intercept: \n', regr.intercept_)
#print('Coefficients: \n', regr.coef_)
regr.score(X,Y)

0.09806996469597307

In [22]:
X

,Respondent,YearsCode,Age,WorkWeekHrs,Assembly,Bash/Shell/PowerShell,C,C#,C++,Clojure,...,"Country_Venezuela, Bolivarian Republic of...",Country_Viet Nam,Country_Yemen,Country_Zambia,Country_Zimbabwe,Extraversion_In real life (in person),Extraversion_Neither,Extraversion_Online,Dependents_No,Dependents_Yes
3,4,3,22.0,80.00,0,0,1,1,1,0,...,0,0,0,0,0,1,0,0,1,0
8,9,12,23.0,32.00,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
12,13,17,28.0,40.00,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
13,14,13,31.0,40.00,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
15,16,10,26.0,40.00,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88324,88877,31,48.0,45.00,0,1,1,0,0,1,...,0,0,0,0,0,0,0,1,0,1
88325,88878,12,26.0,40.00,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
88326,88879,17,34.0,37.75,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
88328,88881,18,37.0,39.00,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [23]:
def train_test(model,x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
    #train_errors, val_errors = [],[]
    
    #interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
    #X_train = interaction.fit_transform(X_train)
    
    preprocessing.StandardScaler()
    model.fit(X_train,y_train,)
    
    y_pred = model.predict(X_test)
    
    intercept = pd.Series(data=model.coef_)
    intercept.name = 'Intercept'
    
    #print('Intercept: \n', model.intercept_)
    #print('Coefficients: \n', model.coef_)

#     train_errors = mean_squared_error(y_train_predict,y_train)
#     val_errors = mean_squared_error(y_val_predict,y_val)
    
    #plt.plot(y_train_predict,y_val_predict)

    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

    df['difference']=df.apply(lambda x: x['Actual']-x['Predicted'],axis=1)
    df['difference_squared'] = df.apply(lambda x: x['difference']**2,axis=1)
    obsv_avg = df['Actual'].mean()
    df['actual_v_mean'] = df.apply(lambda x: (x['Actual']-obsv_avg)**2,axis=1)
    
    variables = pd.DataFrame(data=model.coef_,index=X.columns,columns=['Coefficient'])
    predictor_means = [X_test[x].mean() for x in list(variables.index)]
    variables['Observed_avg'] = predictor_means
    
    #variables.loc['Intercept'] = model.intercept_
    
    performance = {}
    performance['RSS'] = df['difference_squared'].sum()
    performance['TSS'] = df['actual_v_mean'].sum()
    performance['F-Stat'] = ((performance['TSS']-performance['RSS'])/len(x.columns)/
                             (performance['RSS']/len(X_test)-len(x.columns)-1)
                            )
    performance['RSE'] = ((1/(len(X_test)-len(x.columns)))*performance['RSS'])**.5
    performance['MSE'] = mean_squared_error(y_test,y_pred)
    performance['R^2'] = model.score(X_test,y_test)
    performance['observed_mean'] = obsv_avg
    performance['pred_mean'] = df['Predicted'].mean()
    
    variables['Observed_avg'] = predictor_means
    #variables['Observed-Observed_avg'] = [(X_test[x]-variables.loc[x]['Observed_avg']) for x in list(variables.index)]
    
    
    #print(X_test['Respondent']-variables.loc['Respondent']['Observed_avg'])
#     print(X_test.describe())
#     print(predictor_means)
    for predictor in list(variables.index):
        print(predictor)
        #print([value - variables.loc[predictor]['Observed_avg'] for value in X_test[predictor]].sum()/len(X_test))
    print(X_test['Respondent'])
    return df,variables,performance

In [24]:
model,variables,performance = train_test(regr,X,Y)

Respondent
YearsCode
Age
WorkWeekHrs
Assembly
Bash/Shell/PowerShell
C
C#
C++
Clojure
Dart
Elixir
Erlang
F#
Go
HTML/CSS
Java
JavaScript
Kotlin
Objective-C
Other(s):
PHP
Python
R
Ruby
Rust
SQL
Scala
Swift
TypeScript
VBA
WebAssembly
Biracial
Black or of African descent
East Asian
Hispanic or Latino/Latina
Middle Eastern
Multiracial
Native American, Pacific Islander, or Indigenous Australian
South Asian
White or of European descent
OrgSize_1,000 to 4,999 employees
OrgSize_10 to 19 employees
OrgSize_10,000 or more employees
OrgSize_100 to 499 employees
OrgSize_2-9 employees
OrgSize_20 to 99 employees
OrgSize_5,000 to 9,999 employees
OrgSize_500 to 999 employees
OrgSize_Just me - I am a freelancer, sole proprietor, etc.
Gender_Man
Gender_Non-binary, genderqueer, or gender non-conforming
Gender_Woman
Country_Afghanistan
Country_Albania
Country_Algeria
Country_Andorra
Country_Angola
Country_Argentina
Country_Armenia
Country_Australia
Country_Austria
Country_Azerbaijan
Country_Bahrain
Country_B

In [25]:
model.head()

,Actual,Predicted,difference,difference_squared,actual_v_mean
7084,3358.0,-8104.831874,11462.831874,1.313965e+08,1.657809e+10
30576,61071.0,120172.476323,-59101.476323,3.492985e+09,5.047101e+09
21105,21996.0,37415.866921,-15419.866921,2.377723e+08,1.212596e+10
78214,35983.0,147411.063737,-111428.063737,1.241621e+10,9.241160e+09
354,13608.0,11106.703244,2501.296756,6.256485e+06,1.404366e+10


In [26]:
performance

{'RSS': 722775167880288.8,
 'TSS': 794488751819277.5,
 'F-Stat': 4.386631694367993,
 'RSE': 280229.1481554184,
 'MSE': 76752168193.72292,
 'R^2': 0.09026381276610129,
 'observed_mean': 132113.9511521716,
 'pred_mean': 132962.65281925056}

In [27]:
7113-44564.673463

-37451.673463

In [28]:
variables

,Coefficient,Observed_avg
Respondent,0.075086,44564.673463
YearsCode,674.358208,13.123181
Age,1003.355004,31.587246
WorkWeekHrs,149.003280,41.712785
Assembly,754.059043,0.038123
...,...,...
Extraversion_In real life (in person),101.036138,0.629818
Extraversion_Neither,-735.679736,0.095041
Extraversion_Online,634.643598,0.275141
Dependents_No,-2625.965757,0.605288


In [29]:
variables.loc['Respondent']['Observed_avg']

44564.67346288627

In [75]:
ridge = linear_model.Ridge()
X.drop('Respondent',axis=1,inplace=True)

In [44]:
import statsmodels.api as sm

In [215]:
def sm_OLS(X,Y):
    X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    results_summary = results.summary()

    model_performance = {}
    model_performance['MSE'] = results.mse_model
    model_performance['MSE_resid'] = results.mse_resid
    model_performance['RMSE'] = results.mse_resid**.5
    model_performance['R^2'] = results.rsquared
    model_performance['R^2_adjusted'] = results.rsquared_adj
    model_performance['F-stat'] = results.fvalue
    model_performance['AIC'] = results.aic
    model_performance['BIC'] = results.bic
    
    model_performance = pd.DataFrame.from_dict(model_performance,orient='index',columns=['Value'])

    variables_as_html = results_summary.tables[1].as_html()
    variables = pd.read_html(results_as_html, header=0, index_col=0)[0]
    
    outcomes = {}
    outcomes['Observed_values'] = Y
    outcomes['Predicted_values'] = results.predict()
    outcomes['Residuals'] = results.resid
    
    #outcomes['Conf']
    
    outcomes = pd.DataFrame(outcomes)
    #print(results.summary())
    return model_performance,variables,outcomes

In [216]:
sm_performance,sm_variables,outcomes = sm_OLS(X,Y)
#sm_performance['Value'] = sm_performance['Value'].apply(lambda x: round(x,10))

//anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [217]:
sm_performance.loc['MSE_resid']**.5

Value    275443.330592
Name: MSE_resid, dtype: float64

In [221]:
sm_performance.transpose()

,MSE,MSE_resid,RMSE,R^2,R^2_adjusted,F-stat,AIC,BIC
Value,1.867777e+12,7.586903e+10,275443.330592,0.098055,0.094072,24.61844,1.313360e+06,1.315182e+06


In [173]:
variables

,coef,std err,t,P>|t|,[0.025,0.975]
const,598.6090,6176.598,0.097,0.923,-11500.000,12700.000
YearsCode,698.4977,248.295,2.813,0.005,211.835,1185.160
Age,1031.8183,275.220,3.749,0.000,492.384,1571.253
WorkWeekHrs,180.1873,44.436,4.055,0.000,93.093,267.282
Assembly,3441.2592,6868.548,0.501,0.616,-10000.000,16900.000
...,...,...,...,...,...,...
Extraversion_In real life (in person),218.5800,2704.126,0.081,0.936,-5081.547,5518.707
Extraversion_Neither,-290.7050,3675.389,-0.079,0.937,-7494.522,6913.112
Extraversion_Online,670.7340,2957.014,0.227,0.821,-5125.057,6466.525
Dependents_No,-1455.9463,3178.345,-0.458,0.647,-7685.548,4773.655


In [174]:
outcomes

,Observed_values,Residuals,Predicted_values
3,61000.0,-1.734064e+05,234406.441889
8,95179.0,-4.955728e+04,144736.275147
12,90000.0,-1.417045e+05,231704.527346
13,57060.0,-4.981397e+04,106873.970605
15,455352.0,3.063701e+05,148981.941447
...,...,...,...
88324,2000000.0,1.693319e+06,306680.724123
88325,130000.0,-1.292961e+05,259296.051331
88326,82488.0,1.605527e+04,66432.725572
88328,68745.0,-3.265909e+04,101404.088100


In [101]:
variables.sort_values('std err',ascending=False)[:25]

,coef,std err,t,P>|t|,[0.025,0.975]
Country_Andorra,41240.0000,275000.0,0.150,0.881,-497000.0,579000.0
Country_Chad,-89910.0000,274000.0,-0.328,0.743,-627000.0,447000.0
Country_Burkina Faso,-53440.0000,274000.0,-0.195,0.845,-591000.0,484000.0
Country_Tajikistan,-35790.0000,274000.0,-0.131,0.896,-573000.0,501000.0
Country_Angola,-75100.0000,274000.0,-0.274,0.784,-612000.0,462000.0
Country_Malawi,-62650.0000,274000.0,-0.229,0.819,-600000.0,474000.0
Country_Timor-Leste,199000.0000,274000.0,0.726,0.468,-338000.0,736000.0
Country_Togo,-66650.0000,274000.0,-0.243,0.808,-604000.0,470000.0
Country_Liechtenstein,773300.0000,274000.0,2.822,0.005,236000.0,1310000.0
Country_Democratic Republic of the Congo,43620.0000,274000.0,0.159,0.874,-493000.0,581000.0


In [134]:
performance

,ConvertedComp,R-squared:,0.098
Dep. Variable:,,,
Model:,OLS,Adj. R-squared:,0.094
Method:,Least Squares,F-statistic:,24.620
Date:,"Tue, 26 Nov 2019",Prob (F-statistic):,0.000
Time:,14:19:44,Log-Likelihood:,-656470.000
No. Observations:,47083,AIC:,1313000.000
Df Residuals:,46875,BIC:,1315000.000
Df Model:,207,NaN,NaN
Covariance Type:,nonrobust,NaN,NaN


In [62]:
type(a)

statsmodels.iolib.summary.Summary

In [108]:
sm.regression.linear_model.RegressionResults(sm.OLS(Y,X),variables['coef']).fittedvalues

array([234450.591   , 144738.5661  , 231748.678   , ...,  66430.156875,
       101395.618   ,  47962.5446  ])

In [121]:
model = sm.OLS(Y,X)
results = model.fit()

In [142]:
results.f

75869028367.61981